In [1]:
import numpy as np
import pandas as pd

In [2]:
dt = pd.read_csv("Cleveland_80%.csv")
dt.head()

,age,resting_blood_pressure,cholesterol,fasting_blood_sugar,max_heart_rate_achieved,exercise_induced_angina,st_depression,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,st_slope_flat,st_slope_upsloping,target
0,44,120,218,0,115,0,0.0,0,0,0,0,0,0,0,1,0
1,38,145,292,0,130,0,0.0,1,0,1,0,0,1,0,1,0
2,50,140,129,0,135,0,0.0,1,0,0,0,0,1,0,1,0
3,46,120,230,0,150,0,0.0,1,0,1,0,0,1,0,1,0
4,51,110,190,0,120,0,0.0,0,0,1,0,0,1,0,1,0


In [3]:
# checking the shape of dataset
dt.shape

(230, 16)

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,train_test_split

In [5]:
features = dt.drop("target", axis=1)

In [6]:
target = dt["target"]
dt.head()

,age,resting_blood_pressure,cholesterol,fasting_blood_sugar,max_heart_rate_achieved,exercise_induced_angina,st_depression,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,st_slope_flat,st_slope_upsloping,target
0,44,120,218,0,115,0,0.0,0,0,0,0,0,0,0,1,0
1,38,145,292,0,130,0,0.0,1,0,1,0,0,1,0,1,0
2,50,140,129,0,135,0,0.0,1,0,0,0,0,1,0,1,0
3,46,120,230,0,150,0,0.0,1,0,1,0,0,1,0,1,0
4,51,110,190,0,120,0,0.0,0,0,1,0,0,1,0,1,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                        test_size=0.25, stratify=target,
                                                        random_state=5)
print('-----------remodeled-Training Set------------------')
print(X_train.shape)
print(y_train.shape)

-----------remodeled-Training Set------------------
(172, 15)
(172,)


In [8]:
# saving the model
from joblib import dump, load

### Loading the initial federated model

In [9]:
# loading the model from saved file
initial_fed_model = load("initial_fed_model.joblib")
initial_fed_model

RandomForestClassifier(criterion='entropy', max_depth=4, max_features='log2',
                       n_estimators=11, random_state=1)

In [10]:
model1 = initial_fed_model
model1.fit(X_train,y_train)
pred1 = model1.predict(X_test)

acc1 = accuracy_score(y_test,pred1)
print('Test Accuracy_1:', acc1*100)

precision_1 = precision_score(y_test, pred1)
print('Precision_1:', precision_1*100)

recall_1 = recall_score(y_test, pred1)
print('Recall_1:', recall_1*100)

f1_score_1 = f1_score(y_test, pred1)
print('f1_score_1:', f1_score_1*100)

Test Accuracy_1: 86.20689655172413
Precision_1: 73.07692307692307
Recall_1: 95.0
f1_score_1: 82.60869565217392


### Further tuning on existing dataset

In [11]:
# Define the grid of hyperparameters 'params_rf'
params_rf_1 = {'n_estimators': list(range(20,100,5)), 'max_depth': list(range(2,9)),
             'max_features': ['log2','sqrt'], 'criterion': ['gini', 'entropy']}

model1=RandomForestClassifier(random_state=1)

# Instantiate a 5-fold CV grid search object 'grid_rf'
grid_rf_1 = GridSearchCV(estimator=model1, param_grid=params_rf_1, scoring='accuracy', cv=5, n_jobs=-1)

grid_rf_1.fit(X_train, y_train)

# Extract best model from 'grid_rf'
best_model1 = grid_rf_1.best_estimator_

# Extract best hyperparameters from 'grid_rf'
best_hyperparams_1 = grid_rf_1.best_params_
print('Best hyerparameters', best_hyperparams_1)

# Evaluate test set accuracy
pred2 = best_model1.predict(X_test)

test_acc2 = accuracy_score(y_test, pred2)
print('Test Accuracy_2:', test_acc2*100)

precision_2 = precision_score(y_test, pred2)
print('Precision_2:', precision_2*100)

recall_2 = recall_score(y_test, pred2)
print('Recall_2:', recall_2*100)

f1_score_2 = f1_score(y_test, pred2)
print('f1_score_2:', f1_score_2*100)

Best hyerparameters {'criterion': 'gini', 'max_depth': 7, 'max_features': 'log2', 'n_estimators': 85}
Test Accuracy_2: 87.93103448275862
Precision_2: 74.07407407407408
Recall_2: 100.0
f1_score_2: 85.1063829787234


### Saving the tuned model for federated server

In [12]:
dump(best_model1, "client_cleveland_1.joblib")

['client_cleveland_1.joblib']

### Loading the tuned federated model

In [13]:
# loading the model from saved file
fed_model_2 = load("fed_model_2.joblib")
fed_model_2

RandomForestClassifier(criterion='entropy', max_depth=7, max_features='log2',
                       n_estimators=30, random_state=1)

In [14]:
model3 = fed_model_2
model3.fit(X_train,y_train)
pred3 = model3.predict(X_test)

acc3 = accuracy_score(y_test,pred3)
print('Test Accuracy_3:', acc3*100)

precision_3 = precision_score(y_test, pred3)
print('Precision_3:', precision_3*100)

recall_3 = recall_score(y_test, pred3)
print('Recall_3:', recall_1*100)

f1_score_3 = f1_score(y_test, pred3)
print('f1_score_3:', f1_score_3*100)

Test Accuracy_3: 87.93103448275862
Precision_3: 76.0
Recall_3: 95.0
f1_score_3: 84.44444444444444


###  Dataset with newly added sample(Phase 2)

In [15]:
df = pd.read_csv("Cleveland_Full.csv")
df.head()

,age,resting_blood_pressure,cholesterol,fasting_blood_sugar,max_heart_rate_achieved,exercise_induced_angina,st_depression,target,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,st_slope_flat,st_slope_upsloping
0,40,140,289,0,172,0,0.0,0,1,1,0,0,0,1,0,1
1,49,160,180,0,156,0,1.0,1,0,0,1,0,0,1,1,0
2,37,130,283,0,98,0,0.0,0,1,1,0,0,0,0,0,1
3,48,138,214,0,108,1,1.5,1,0,0,0,0,0,1,1,0
4,54,150,195,0,122,0,0.0,0,1,0,1,0,0,1,0,1


In [16]:
df.shape

(288, 16)

In [17]:
X = df.drop("target", axis=1)

In [18]:
y = df["target"]

In [19]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, 
                                                        test_size=0.25, stratify=y,
                                                        random_state=5)
print('-----------remodeled-Training Set------------------')
print(X_train2.shape)
print(y_train2.shape)

-----------remodeled-Training Set------------------
(216, 15)
(216,)


### Using the federated models parameters to evaluate dataset with new samples

In [28]:
# loading the model from saved file
fed_model_3 = load("fed_model_3.joblib")
print(fed_model_3)

model4 = fed_model_3
model4.fit(X_train2,y_train2)
pred4 = model4.predict(X_test2)

acc4 = accuracy_score(y_test2,pred4)
print('Test Accuracy_4:', acc4*100)

precision_4 = precision_score(y_test2, pred4)
print('Precision_4:', precision_4*100)

recall_4 = recall_score(y_test2, pred4)
print('Recall_4:', recall_4*100)

f1_score_4 = f1_score(y_test2, pred4)
print('f1_score_4:', f1_score_4*100)

RandomForestClassifier(criterion='entropy', max_depth=7, max_features='log2',
                       n_estimators=35, random_state=1)
Test Accuracy_4: 97.22222222222221
Precision_4: 96.0
Recall_4: 96.0
f1_score_4: 96.0
